In [50]:
### import required libraries

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# import tweepy
# from tweepy import Stream
# from tweepy import OAuthHandler
# from tweepy.streaming import StreamListener
# import json

In [51]:
### set date range for weekly report
date_range_high = datetime.today().date()
date_range_low = datetime.today().date() - timedelta(days=7)

In [52]:
user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'

#### Reuters M&A News

In [6]:
### webscrape Reuters M&A News

date_filter = date_range_high
page_number = 1
reuters_news_dict = {}
reuters_news_url = "https://www.reuters.com/news/archive/mergersnews?view=page&page={}&pageSize=10"

while date_filter >= date_range_low:

    reuters_raw = requests.get(reuters_news_url.format(page_number), headers={'User-Agent': user_agent})
    reuters_bs4 = BeautifulSoup(reuters_raw.content)

    reuters_news_block = reuters_bs4.find_all('div', class_ = 'column1 col col-10')
    reuters_news_list  = reuters_news_block[0].find_all('h3', class_ = 'story-title')
    reuters_timestamp_list = reuters_news_block[0].find_all('span', class_ = 'timestamp')

    for i in range(len(reuters_news_list)):
        title = reuters_news_list[i].get_text()
        title = title.split('\n\t\t\t\t\t\t\t\t')[1]

        a_date = reuters_timestamp_list[i].get_text()
        if 'am' in a_date or 'pm' in a_date:
            a_date = datetime.today().date()
        else:
            a_date = datetime.strptime(a_date, "%b %d %Y").date()

        reuters_news_dict[title] = a_date

    date_filter = a_date
    page_number += 1

In [7]:
reuters_news_dict = {title: date for title, date in reuters_news_dict.items() if date >= date_range_low}

In [8]:
len(reuters_news_dict)

111

In [9]:
reuters_news_dict

{"UPDATE 2-UniCredit unlikely to pursue deal for Russia's Otkritie, source says": datetime.date(2022, 1, 24),
 "UPDATE 1-Kohl's shares soar on takeover interest from Sycamore, Acacia": datetime.date(2022, 1, 24),
 'UPDATE 3-M&C Saatchi rejects improved offer from top shareholder': datetime.date(2022, 1, 24),
 "Kohl's shares soar as Sycamore, Acacia show takeover interest": datetime.date(2022, 1, 24),
 'UPDATE 2-KKR buys Sparta, Raleigh bike maker Accell for $1.77 bln': datetime.date(2022, 1, 24),
 "Brazil's sliding stock market makes M&A targets": datetime.date(2022, 1, 24),
 "SPAC linked to Trump's venture outperforms others in sector": datetime.date(2022, 1, 24),
 'UPDATE 1-Luxury goods group Kering to sell watches division': datetime.date(2022, 1, 24),
 "Malaysia's Raya Airways denies acquisition talks with AirAsia": datetime.date(2022, 1, 24),
 'Luxury goods group Kering to sell Girard-Perregaux and Ulysse Nardin divisions': datetime.date(2022, 1, 24),
 "Saudi Arabia's SABIC Agri-N

#### Seeking Alpha M&A News

In [56]:
### webscrape Seeking Alpha M&A News

options = Options()
# options.add_argument("start-maximized")
# options.add_argument('--headless')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

s = Service('/Users/babyghost/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver')
driver = webdriver.Chrome(service= s, options=options); 

# driver = webdriver.Chrome(service = ChromeDriverManager().install(), options = options) ### figure out your chromedriver path

In [18]:
article_list = []
formatted_date_list = []

driver.get('https://seekingalpha.com/market-news/m-a?page=1')

article_block = driver.find_elements(By.XPATH, "//*[@id='content']/div/div[2]/div/div[2]/section/div/div/div/div[2]")[0].text.split('\n')

for i, j in enumerate(article_block):
    if i%2 == 0:
        article_list.append(j)

date_list = driver.find_elements(By.XPATH, "//*[@id='content']/div/div[3]/div/div[2]/section/div/div/div/div[2]/article[1]/div/div/footer/span[2]")
date_list = [i.text for i in date_list] 

for a_date in date_list:
    if 'Today' in a_date:
        a_date = datetime.today().date()
    elif 'Yesterday' in a_date:
        a_date = datetime.today().date() - timedelta(days = 1)
    elif len(a_date.split(' ')) == 3:
        a_date = a_date + ' ' + str(datetime.today().date().year)
        a_date = datetime.strptime(a_date, "%a, %b. %d %Y").date()
    else:
        a_date = datetime.strptime(a_date, "%a, %b. %d, %Y").date()
    
    formatted_date_list.append(a_date)

IndexError: list index out of range

In [17]:
date_list

[]

In [14]:
seeking_alpha_news_dict = dict(zip(article_list, formatted_date_list))

In [15]:
seeking_alpha_news_dict

{}

#### theMiddleMarket M&A News

In [42]:
ma_news_url = 'https://www.themiddlemarket.com/latest-news'
driver.get(ma_news_url)

latest_news = driver.find_elements(By.XPATH,"/html/body/main/div/div/div/div[1]/div/div")
latest_news = latest_news[0].text.split('\n')

ma_article_list = [i for count, i in enumerate(latest_news) if count%2 == 0 ] 
ma_date_list = [i for count, i in enumerate(latest_news) if count%2 != 0 ] 
theMiddleMarket_news_dict =  dict(zip(ma_article_list, ma_date_list))

In [43]:
theMiddleMarket_news_dict = {key: datetime.strptime(value.title(), '%B %d, %Y').date() for key, value in theMiddleMarket_news_dict.items()}
theMiddleMarket_news_dict = {title: date for title, date in theMiddleMarket_news_dict.items() if date >= date_range_low}

In [62]:
# theMiddleMarket_news_dict

#### New York Times Mergers News

In [57]:
nyt_url = "https://www.nytimes.com/topic/subject/mergers-acquisitions-and-divestitures"
driver.get(nyt_url)
nyt_news_raw = driver.find_elements(By.XPATH, '//*[@id="collection-Mergers, Acquisitions and Divestitures"]/div[1]/div')[0].text.split('\n')

In [68]:
index_list = []
date_list = []
for count, text in enumerate(nyt_news_raw):
    try:
        a_date = datetime.strptime(text, "%b. %d, %Y")
        date_list.append(a_date)
        index_list.append(count -2)
    except:
        pass
    
title_list = [nyt_news_raw[i] for i in index_list]
nyt_ma_news_dict = dict(zip(title_list, date_list))

#### Monitor Twitter Keyword

In [7]:
api_doc = pd.read_excel('jh_twitter_api_doc.xlsx')

consumer_key = api_doc['key'][0]
consumer_secret = api_doc['secret'][0]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [9]:
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [11]:
# session.set('request_token', auth.request_token['oauth_token'])

In [ ]:
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)